In [1]:
print("hello world")

hello world


In [4]:
%pwd

'd:\\Projects\\AI-Medical-recommendation'

In [ ]:
# import os
# os.chdir("../")

In [5]:
pip show langchain

Name: langchain
Version: 0.3.21
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: d:\Projects\AI-Medical-recommendation\venv\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extarct data from the Pdf File
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob = "*.pdf", loader_cls = PyPDFLoader)
    documents =loader.load()
    return documents

In [8]:
%pwd

'd:\\Projects\\AI-Medical-recommendation'

In [9]:
extracted_data = load_pdf_file(data = 'Data/')

In [9]:
# extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter =RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap= 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of Text Chunks", len(text_chunks))

length of Text Chunks 825


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\nprat\AppData\Local\Temp\ipykernel_16760\3605476162.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Projects\AI-Medical-recommendation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
query_result = embeddings.embed_query("Hello World")
print("Length ", len(query_result))

Length  384


In [18]:
# query_result

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
import os

load_dotenv(dotenv_path=".env")  # Ensure the correct path
api_key = os.getenv("DEEPSEEK_API_KEY")
print(api_key)

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


In [29]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
DEEPSEEK_API_KEY = os.environ.get('DEEPSEEK_API_KEY')

In [35]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "aimedication"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '1ea34587cef1f1036d3329111e466bbd', 'date': 'Tue, 01 Apr 2025 10:09:57 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [30]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY

In [ ]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [37]:
#Load the Existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [38]:
docsearch

In [39]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [42]:
retrieved_docs = retriver.invoke("how to cure thyroid ?")

In [43]:
retrieved_docs

[Document(id='652e86ab-727b-4420-b73d-c198064d69c6', metadata={'author': 'LAKSHMI CHANDRA MISHRA', 'creationdate': '2007-08-25T22:24:32+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2007-08-25T22:24:32+02:00', 'page': 149.0, 'page_label': '150', 'producer': 'Adobe Acrobat 6.0', 'source': 'Data\\Scientific_Basis_for_Ayurvedic_Therapies.pdf', 'subject': 'Front Matter', 'title': 'SCIENTIFIC BASIS FOR AYURVEDIC THERAPIES', 'total_pages': 600.0}, page_content='internally.\nFew herbal products are already available for the treatment of thyroid dysfunctions; the\nfood supplements, \n \nVimbadi \n \noil (with sesame oil) can be applied four times/day on the\ngoiter.\n \n2\n \n \n \nYograj guggul\n \n (0.5 to 2.25 g/day) and \n \nAshwagandha \n \npowder (\n \nchurn\n \n) (1 to 6 g/day)\nare also used in the treatment of hypothyroidism. A study on Liv-52, an Ayurvedic liver\ntonic, has revealed that it stimulates thyroid hormone secretion, particularly the T\n \n3\n \n.\n \n11\n \n In'),
 

In [44]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an AI health and wellness consultant specializing in personalized health, fitness, and travel safety recommendations. Your role is to provide medically accurate, natural, and scientific advice based on the user's concerns."

   " If the user asks about:"  
    "1. General Health & Fitness - Recommend dietary habits, exercises, and lifestyle improvements."  
    "2. Travel Health Precautions - If the user mentions allergies, food restrictions, weather conditions, or environmental factors, provide:"  
    "- Recommended fruits, vegetables, herbal remedies, and medications for safety and adaptation."  
    "- Precautionary measures like hydration, breathing exercises, and clothing choices."  
    "3. Natural Remedies & Alternative Medicine - Suggest Ayurvedic, herbal, or holistic treatments for improving well-being."  
    "4. Climate & Environmental Adaptation - Guide users on adjusting to different weather conditions (hot, cold, humid, dry, etc.). " 
    "5. Food & Nutrient Advice - Recommend immune-boosting foods, vitamins, and hydration tips based on health concerns."  

    "If the question is outside these topics, respond with:"  
    "Sorry, I can't help you with this :("  

    "Do not provide answers for unrelated topics like finance, technology, entertainment, history, or politics. If the query is unclear, ask a clarifying question before proceeding." 
    "\n\n"
    "{context}" 


)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": "What is acne ?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}